# NBA Player Performance Predictor Prototype

In [355]:
from nba_api.stats.endpoints import leaguedashplayerstats, leaguestandings
from nba_api.stats.static import teams, players
import pandas as pd
from rapidfuzz import process, fuzz

season_year = '2024-25'
player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season_year, league_id_nullable='00')
league_standings = leaguestandings.LeagueStandings(season=season_year, league_id='00').get_data_frames()[0]
player_stats_df_csv = player_stats.get_data_frames()[0]
nba_teams = teams.get_teams()

player_stats_df_csv.to_csv('nba_player_stats.csv', index=False)

nba_players = players.get_active_players()

drop_teams_cols = ['strLongHomeStreak', 'strLongRoadStreak', 'strCurrentHomeStreak', 
    'strCurrentRoadStreak', 'strCurrentStreak',
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec',
    'PreAS', 'PostAS',
    'vsEast', 'vsAtlantic', 'vsCentral', 'vsWest', 'vsPacific', 'vsSouthwest', 'vsSoutheast', 'vsNorthwest',
    'OT', 'ThreePTSOrLess', 'TenPTSOrMore', 'OppOver500',
    'AheadAtHalf', 'BehindAtHalf', 'TiedAtHalf',
    'AheadAtThird', 'BehindAtThird', 'TiedAtThird',
    'Score100PTS', 'OppScore100PTS',
    'LeadInFGPCT', 'LeadInReb', 'FewerTurnovers', 'ConferenceGamesBack', 'DivisionGamesBack', 'ClinchIndicator']

league_standings.reset_index(drop=True, inplace=True)
league_standings.drop(columns=drop_teams_cols, inplace=True)

In [356]:
# Player search implementation

def searchPlayers(df, usr_input, threshold=75, limit=None):
    # Get list of player names (lowercased)
    names = df['PLAYER NAME'].str.lower().tolist()
    
    # Get top fuzzy matches with partial_ratio scorer
    matches = process.extract(usr_input.lower(), names, limit=limit, scorer=fuzz.partial_ratio)
    
    # Filter matches by threshold score
    top_matches = [m for m in matches if m[1] > threshold]
    
    # Get original DataFrame rows for those matches
    matched_rows = [df.iloc[m[2]] for m in top_matches]
    
    return pd.DataFrame(matched_rows)
    
# Team search implementation

def searchByTeams(nba_teams_df, usr_input, threshold=75, limit=None):
    # Extract the team names as a list, lowercase for fuzzy matching
    names = nba_teams_df['TEAM'].str.lower().tolist()
    
    # Use fuzzy matching with partial_ratio scorer on user input
    matches = process.extract(usr_input.lower(), names, limit=limit, scorer=fuzz.partial_ratio)
    
    # Filter matches above the threshold
    top_matches = [m for m in matches if m[1] > threshold]
    
    # Retrieve matching rows from the original DataFrame by index
    matched_rows = [nba_teams_df.iloc[m[2]] for m in top_matches]
    
    # Return as a DataFrame
    return pd.DataFrame(matched_rows)

In [357]:
# Sorted columns and displayed search feature for players

player_stats_df = pd.read_csv("nba_player_stats.csv")
team_stats_df = pd.read_csv("nba_team_stats.csv")

drop_cols = ['NICKNAME',
    'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK',
    'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
    'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
    'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
    'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK',
    'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
    'BLKA',"WNBA_FANTASY_PTS", "WNBA_FANTASY_PTS_RANK",
    'TEAM_COUNT']

player_stats_df.head()
player_stats_df.loc[player_stats_df["PLAYER_NAME"].str.startswith("L")]
player_stats_df.insert(loc=3, column="Player Image", value=player_stats_df.index.to_series().apply(lambda pid: f"https://cdn.nba.com/headshots/nba/latest/1040x760/{player_stats_df.iloc[pid]["PLAYER_ID"]}.png"))
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
player_stats_df.sort_values(by=("PLAYER_NAME"))
nba_teams = pd.DataFrame(nba_teams)

abbr_to_full = nba_teams.set_index("abbreviation")["full_name"]
player_stats_df.insert(loc=5, column="TEAM", value=player_stats_df["TEAM_ABBREVIATION"].map(abbr_to_full))
player_stats_df.drop(columns=drop_cols, inplace=True)
player_stats_df.rename(columns=lambda x: x.replace("_", " ").upper() if len(x) > 3 or not x.isalnum() else x, inplace=True)
player_stats_df.sort_values(by=["TEAM", "PLAYER NAME"], inplace=True)
player_stats_df.reset_index(drop=True, inplace=True)
player_stats_df["AGE"] = player_stats_df["AGE"].astype(int)

In [358]:
league_standings.sort_values(by=['Conference', 'WINS'], ascending=False, inplace=True)
east_standings = league_standings[league_standings["Conference"] == "East"]
west_standings = league_standings[league_standings["Conference"] == "West"]
east_standings.reset_index(drop=True, inplace=True)
west_standings.reset_index(drop=True, inplace=True)

In [359]:
league_standings

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,Division,DivisionRecord,DivisionRank,WINS,LOSSES,WinPCT,LeagueRank,Record,HOME,ROAD,L10,Last10Home,Last10Road,LongHomeStreak,LongRoadStreak,LongWinStreak,LongLossStreak,CurrentHomeStreak,CurrentRoadStreak,CurrentStreak,ClinchedConferenceTitle,ClinchedDivisionTitle,ClinchedPlayoffBirth,EliminatedConference,EliminatedDivision,PointsPG,OppPointsPG,DiffPointsPG
1,00,22024,1610612760,Oklahoma City,Thunder,West,39-13,1,Northwest,12-4,1,68,14,0.829,1.0,68-14,35-6,32-8,8-2,8-2,9-1,12,11,15,2,1,3,4,1,1,1,0,0,121.0,108.0,13.0
3,00,22024,1610612745,Houston,Rockets,West,31-21,2,Southwest,13-3,1,52,30,0.634,2.0,52-30,29-12,23-17,6-4,8-2,6-4,7,-8,9,6,-1,-2,-3,0,1,1,0,0,114.0,110.0,5.0
4,00,22024,1610612747,Los Angeles,Lakers,West,36-16,3,Pacific,12-4,1,50,32,0.610,NaN,50-32,31-10,19-22,6-4,7-3,4-6,9,-5,8,4,2,-1,-1,0,1,1,0,0,113.0,112.0,1.0
7,00,22024,1610612743,Denver,Nuggets,West,32-20,4,Northwest,8-8,2,50,32,0.610,NaN,50-32,26-15,24-17,5-5,4-6,5-5,8,6,9,4,1,2,3,0,0,1,0,0,121.0,117.0,4.0
8,00,22024,1610612746,LA,Clippers,West,29-23,5,Pacific,9-7,2,50,32,0.610,NaN,50-32,30-11,20-21,9-1,9-1,7-3,9,4,8,3,5,3,8,0,0,1,0,0,113.0,108.0,5.0
11,00,22024,1610612750,Minnesota,Timberwolves,West,33-19,6,Northwest,11-5,3,49,33,0.598,NaN,49-33,25-16,24-17,8-2,8-2,8-2,5,4,8,4,5,1,3,0,0,1,0,0,114.0,109.0,5.0
13,00,22024,1610612744,Golden State,Warriors,West,29-23,7,Pacific,5-11,3,48,34,0.585,3.0,48-34,24-17,24-17,7-3,6-4,8-2,7,6,7,5,-3,6,-1,0,0,0,0,0,114.0,110.0,3.0
14,00,22024,1610612763,Memphis,Grizzlies,West,27-24,8,Southwest,11-5,2,48,34,0.585,NaN,48-34,26-15,22-19,4-6,4-6,5-5,8,3,6,4,1,-1,1,0,0,0,0,0,122.0,117.0,5.0
17,00,22024,1610612758,Sacramento,Kings,West,26-26,9,Pacific,5-11,4,40,42,0.488,NaN,40-42,20-21,20-21,5-5,4-6,3-7,7,-7,7,6,1,3,1,0,0,0,0,0,116.0,115.0,0.0
18,00,22024,1610612742,Dallas,Mavericks,West,23-29,10,Southwest,8-8,3,39,43,0.476,4.0,39-43,22-18,17-25,4-6,3-7,3-7,-5,-6,7,5,1,-3,-1,0,0,0,0,0,114.0,115.0,-1.0


In [360]:
usrInput = input("Search for an NBA Player: ")
searchPlayers(player_stats_df, usrInput)

,PLAYER ID,PLAYER NAME,PLAYER IMAGE,TEAM ID,TEAM,TEAM ABBREVIATION,AGE,GP,W,L,W PCT,MIN,FGM,FGA,FG PCT,FG3M,FG3A,FG3 PCT,FTM,FTA,FT PCT,OREB,DREB,REB,AST,TOV,STL,BLK,PF,PFD,PTS,PLUS MINUS,NBA FANTASY PTS,DD2,TD3
254,2544,LeBron James,https://cdn.nba.com/headshots/nba/latest/1040x760/2544.png,1610612747,Los Angeles Lakers,LAL,40,70,44,26,0.629,2444.446667,651,1270,0.513,149,396,0.376,259,331,0.782,72,474,546,575,260,70,39,99,265,1710,-54,3294.7,32,10
245,1642355,Bronny James,https://cdn.nba.com/headshots/nba/latest/1040x760/1642355.png,1610612747,Los Angeles Lakers,LAL,20,27,17,10,0.630,181.243333,21,67,0.313,9,32,0.281,11,14,0.786,4,14,18,22,13,9,3,13,10,62,-71,139.6,0,0
